In [ ]:
from deepface import DeepFace
import os
import cv2
import json

'''
Goal is to use deepface (and it's parameters) to handle the process of 
classifing the following:
    1. Facial detection and skewing
    2. create facial feature vector
    3. Race classification
    4. (currently) classify emotion
Our next step will be to train our own culturally aware emotion classifier
by using the outputs of the deepface model (its labels and confidence rating) 
'''

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


"\nGoal is to use deepface (and it's parameters) to handle the process of \nclassifing the following:\n    1. Facial detection and skewing\n    2. create facial feature vector\n    3. Race classification\n    4. (currently) classify emotion\nOur next step will be to train our own culturally aware emotion classifier\nby using the outputs of the deepface model (its labels and confidence rating) \n"

In [ ]:
import cv2
'''
function wrapper to vectorize face of image
This function detects the face from the image
and returns the face vector for that image
    
SRC: https://github.com/serengil/deepface/blob/master/deepface/detectors/FaceDetector.py
'''
def vectorizeFace(image_path, backend):
    return DeepFace.detectFace(image_path, detector_backend=backend)

In [ ]:
'''
returns the input image (from path) cropped and aligned
That is, ready for input into vectorizer

SRC: https://github.com/serengil/deepface/blob/master/deepface/commons/functions.py
'''
def cropAlignFace(image_path, target_size, enforce_detection, detector_backend):
    return  functions.preprocess_face(
            img=image_path,
            target_size=target_size,
            enforce_detection = enforce_detection,
            detector_backend = detector_backend)



In [ ]:
'''
    input: path to an image, backend dectector model = ['retinaface', 'mtcnn', 'opencv', 'ssd', 'dlib']
    returns the following in a json-like dictionary:
    emotion confidence: {angry, disgust, fear, happy, sad, suprise, neutral}
    dominant emotion: the emotion with highest value from emotion confidence
    race confidence: {asian, indian, black, white, middle eastern, latino hispanic}
    dominant_race: the race with highest value from race confidence
'''
def analyzeRaceEmotion(image_path, backend='opencv' ):
    return DeepFace.analyze(img_path=image_path, actions = ['emotion'], enforce_detection=False, detector_backend=backend)

In [ ]:
#! pip install torch

In [ ]:
'''
    Turn fer2013 dataset into object that can be processed.
    Note: This stores the entire dataset into memory (~300MB)
    Modified by Joshua Tapp
    Original Source: https://medium.com/analytics-vidhya/read-fer2013-face-expression-recognition-dataset-using-pytorch-torchvision-9ff64f55018e
'''
from torch import torch
from torch.utils.data import Dataset
import numpy as np 

class FER2013Dataset_Alternative(Dataset):
    """Face Expression Recognition Dataset"""
    
    def __init__(self, file_path):
        """
        Args:
            file_path (string): Path to the csv file with emotion, pixel & usage.
        """
        self.file_path = file_path
        self.classes = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral') # Define the name of classes / expression
        
        with open(self.file_path) as f: # read all the csv using readlines
            self.data = f.readlines()
            
        self.total_images = len(self.data) - 1 #reduce 1 for row of column

    def __len__(self):  
        return self.total_images
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        with open(fer_path) as f:
            emotion, img, usage = self.data[idx + 1].split(",") #plus 1 to skip first row (column name)
            
        emotion = int(emotion) # just make sure it is int not str
        img = img.split(" ") # because the pixels are seperated by space
        img = np.asarray(img, dtype=np.uint8).reshape(48,48)

        sample = {'image': img, 'emotion': emotion}
        
        return sample


In [ ]:
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace
from deepface.commons import functions

import matplotlib.pyplot as plt
import numpy as np



In [ ]:
fer_path = '/work/fer2013.csv'
dataset = FER2013Dataset_Alternative(fer_path)
image = dataset[1000]

# test getting the right object
print('image:\n', image['image'])
print('emotion:\t', image['emotion'])

image:
 [[ 47  42  40 ...  25  29  56]
 [ 40  39  52 ...  21  32  65]
 [ 47  82 103 ...  48  40  47]
 ...
 [199 246 236 ... 117 107 100]
 [238 239 237 ... 163 139 110]
 [234 239 230 ... 189 162 147]]
emotion:	 6


In [ ]:
'''
Function to convert the image data from fer2013
into 48x48 images using the PIL lib.
    Input: image as np array from the dataset object
    Output: 48x48 B&W png file

from PIL import Image
def convertFer2013Image(imgArray):
    return Image.fromarray(imgArray)
'''

'\nFunction to convert the image data from fer2013\ninto 48x48 images using the PIL lib.\n    Input: image as np array from the dataset object\n    Output: 48x48 B&W png file\n\nfrom PIL import Image\ndef convertFer2013Image(imgArray):\n    return Image.fromarray(imgArray)\n'

In [ ]:

imgArray = image['image']
converted_image = convertFer2013Image(imgArray)
converted_image

NameError: name 'convertFer2013Image' is not defined

In [ ]:
type(converted_image)

NameError: name 'converted_image' is not defined

In [ ]:
#for emotion conversion
def convertFer2013Emotion(num):
    emotions = {
        0 : 'angry', 
        1 : 'disgust', 
        2 : 'fear',
        3 : 'happy',
        4 : 'sad',
        5 : 'surprise',
        6 : 'neutral'
    }
    return emotions[num]

#for ethnicity conversion
def convertUTKEthnicity(num):
    race_dict = {
    '0': "white",
    '1': "black",
    '2': "asian",
    '3': "indian",
    '4': "other"
    }
    return race_dict[num]


In [ ]:
convertFer2013Emotion(image['emotion'])

'neutral'

In [ ]:
'''
Test our DeepFace model on the fer2013 dataset.
'''
size = len(dataset)
file_path_format = format('/work/CulturalEmotionClassifer/fer2013_images/{num:n}.png')

for i in range(size):
    filename = file_path_format.format(num = i)
    imgArray = dataset[i]['image']
    img = convertFer2013Image(imgArray)
    img.save(filename)


NameError: name 'convertFer2013Image' is not defined

In [ ]:
# in order list of all the emotions (as lowercase strings) for the images saved in fer2013_images
fer2013_emotions = []
size = len(dataset)
for i in range(size):
    line = dataset[i]
    fer2013_emotions.append(convertFer2013Emotion(line['emotion']))


In [ ]:
test =  {
    'white':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'black':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    }
}

for k,v in test:
    print(k)
    print(test[k]['tp'])

ValueError: too many values to unpack (expected 2)

In [ ]:
def calc_recall (stats, k):

    tp = stats[k]['tp']
    fn = stats[k]['fn']

    #divide by zero check
    if (tp+fn) > 0:
        return tp/(tp+fn)
    else:
        return "N/A"

def calc_precision(stats, k): 
#values of dict 

    tp = stats[k]['tp']
    fp = stats[k]['fp']

    #divide by zero check
    if( (tp+fp) > 0):
        return tp/(tp+fp)
    else:
        return "N/A"

def calc_f1 (stats, k):

    precision = calc_precision(stats, k)
    recall = calc_recall (stats, k)

        #can only return f1 score if precision/recall did not divide by zero
    if ( isinstance(precision,int) or isinstance(precision,float) ) and ( isinstance(recall,int) or isinstance(recall,float) ):
        return (2*(precision*recall)) / (precision + recall)
    else:
        return ("N/A")

def print_metrics (stats):

    for k in stats:

        print("")
        print(k)
        print ("precision: ", calc_precision(stats, k))
        print ("recall: ", calc_recall(stats, k))
        print ("f1: ", calc_f1(stats, k))
        



In [ ]:
#code for ethnicity testing

from deepface.detectors import FaceDetector


directory = '../UTKFace'
#all of the .jpg files from the 'UTKFace' Directory
#can use these indicies for indicies of other data objects
cropped_pictures = os.listdir(directory)

#will be a nested list of photos stored such as:
#[[filename1, age1, gender1, race1],[filename2, age2, gender2, race2]...]
pictures_list = []

#store results of the deepface algorithm for races
results = []

#correct number of classifications
correct = 0

#current iterations
count = 0

#number of face detections
total_faces = 0

#count number of images
total=0

obj = 0



#dictionaries for TP/FP/TN/FN
stats = {
    'white':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'black':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'asian':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'indian':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'other':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    }
}



for files in cropped_pictures:

    total += 1

    #split by underscores
    data_items = files.split("_")
    
    pictures_list.append([files, data_items[0],data_items[1],data_items[2]])

    image_path =  directory+"/"+files

    count += 1

    temp = DeepFace.analyze(img_path = image_path, detector_backend = "skip", actions = ['race'], enforce_detection=False )
    face_detector = FaceDetector.build_model('opencv')

    #not sure why this exception isn't working
    try:
        detected_face,img_region = FaceDetector.detect_face(face_detector, "opencv", img = functions.load_image(image_path))
        # total_faces += 1
    except:
        print("face not detected for ethnicity. File:", image_path)

    if (detected_face is not None):
        total_faces += 1

    obj = temp

    classified_race = obj['dominant_race']

    #these are the only valid races in the photo set
    if classified_race not in ['white', 'black', 'asian', 'indian']:

        #have to set to other (like Hispanic, Latino, Middle Eastern)
        classified_race = 'other'

    #append the resulting race from deepface into the results list
    results.append(classified_race)

    correct_race = convertUTKEthnicity(data_items[2])

    #if correct classification
    if correct_race == classified_race:
        print("correct!")
        correct += 1

        #increment tp for correct race
        stats[classified_race]['tp'] += 1

        for k in stats:
            if k is not classified_race :
                
                #increment true negative for all other classifications
                stats[k]['tn'] += 1
    
    #incorrect classification
    else:

        #increment fp for the model's classified race
        stats[classified_race]['fp'] += 1
        
        #increment fn for the true race from dataset
        stats[correct_race]['fn'] += 1

        for k in stats:
            if k not in (classified_race, correct_race):
                #increment true negative for all other classifications
                stats[k]['tn'] += 1

    if (count == 1000):
        break

    print(count)

print("Number of correct classifications:", correct)



print ("total accuracy:", correct/total)

print_metrics(stats)
    

race_model_single_batch.h5 will be downloaded...
Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/race_model_single_batch.h5
To: /root/.deepface/weights/race_model_single_batch.h5
100%|██████████| 537M/537M [00:10<00:00, 53.5MB/s]


NameError: name 'convertUTKEthnicity' is not defined

In [ ]:
correct_preds = 0


emotion_stats = {
    'angry':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'disgust':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'fear':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'happy':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'sad':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'surprise':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    },
    'neutral':{
        'tp':0,
        'tn':0,
        'fp':0,
        'fn':0
    }
}


for i in range(1000):
    # grab the right image , /work/CulturalEmotionClassifer/fer2013_images/%i.png
    img = "/work/CulturalEmotionClassifer/fer2013_images/{}.png".format(i)
    
    # run image thru analyzeRaceEmotion(image)
    # ['retinaface', 'mtcnn', 'opencv', 'ssd', 'dlib']
    predicted_emotion = analyzeRaceEmotion(img)['dominant_emotion']
    
    # grab label fer2013_emotions[i]
    actual_emotion = fer2013_emotions[i]

    # does label == predict
    if actual_emotion == predicted_emotion:
        # if yes, count++
        correct_preds = correct_preds + 1

        #increment tp for correct race
        emotion_stats[predicted_emotion]['tp'] += 1

        for k in emotion_stats:
            if k is not predicted_emotion :
                
                #increment true negative for all other classifications
                emotion_stats[k]['tn'] += 1
    
    #incorrect classification
    else:

        #increment fp for the model's classified race
        emotion_stats[predicted_emotion]['fp'] += 1
        
        #increment fn for the true race from dataset
        emotion_stats[actual_emotion]['fn'] += 1

        for k in emotion_stats:
            if k not in (predicted_emotion, actual_emotion):
                #increment true negative for all other classifications
                emotion_stats[k]['tn'] += 1
   
accuracy = correct_preds / size
print("")
print(accuracy)
print_metrics(emotion_stats)


Action: race: 100%|██████████| 2/2 [00:00<00:00,  4.50it/s]
0.02496725833867417



NameError: name 'print_metrics' is not defined

The performance of our DeepFace (default) model on emotion classifying on the fer2013 dataset (converted to png images) is **0.8317775238944465**. It is possible that changes to the backend of the model would increase accuracy. However, this is our fastest computing model and it still took almost 5 hours to compute the results on the ~38000 images

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread('/work/CulturalEmotionClassifer/fer2013_images/4.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # use this to color correct plt.imshow(img)
emote_4 = fer2013_emotions[4]
print('labeled emotion:', emote_4)
# ['retinaface', 'mtcnn', 'opencv', 'ssd', 'dlib']
pred_emote_4 = analyzeRaceEmotion(img, 'dlib')
print('predicted emotion:', pred_emote_4['dominant_emotion'] )

plt.imshow(img)
plt.show()

labeled emotion: neutral
Action: emotion:   0%|          | 0/2 [00:00<?, ?it/s]


ModuleNotFoundError: No module named 'dlib'

In [ ]:
# Race and Emotion


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7994f515-40bb-4330-b195-ec7f31bf201d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>